In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [2]:
from mriqc_learn.datasets import load_dataset
from mriqc_learn.models import preprocess as pp
from mriqc_learn.models.production import init_pipeline
from mriqc_learn.model_selection import split

## Load some data
We first load the ABIDE dataset, one of the default datasets distributed with MRIQC-learn

In [3]:
(train_x, train_y), (_, _) = load_dataset(split_strategy="none")
train_x["site"] = train_y.site

Let's pick the ratings from "rater_3" and binarize the three categories into only two.
We can also see that the dataset is unbalanced.

In [4]:
train_y = train_y[["rater_3"]].values.squeeze()
print(f"Discard={100 * (train_y == -1).sum() / len(train_y)}")
print(f"Doubtful={100 * (train_y == 0).sum() / len(train_y)}")
print(f"Accept={100 * (train_y == 1).sum() / len(train_y)}")
train_y[train_y == 0] = -1
train_y += 1
train_y[train_y > 0] = 1

Discard=14.168937329700272
Doubtful=1.5440508628519527
Accept=84.28701180744777


Let's print out a pretty view of the data table:

In [5]:
train_x

,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,icvs_gm,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,site
0,0.383747,3.259968,0.609668,181.619858,3.944888,3.959924,4.039157,3.835584,0.199774,0.449138,...,1000.013428,189965.0,908.938904,1079.413428,51.778980,0.225944,0.525072,0.540801,0.540213,PITT
1,0.574080,2.279440,0.606361,172.500031,3.992397,3.877495,4.173095,3.926602,0.203301,0.429628,...,1000.033569,187992.0,901.788293,1120.833569,67.136932,0.223374,0.521399,0.560238,0.571425,PITT
2,0.314944,3.998569,0.577123,273.688171,4.016382,4.066009,4.092888,3.890248,0.201591,0.446495,...,1000.015198,188213.0,913.847803,1067.003662,46.623932,0.233414,0.531020,0.556496,0.612655,PITT
3,0.418505,3.050534,0.571343,237.531143,3.601741,3.629409,3.627568,3.548246,0.190612,0.468255,...,1000.005981,146722.0,872.409717,1083.139264,63.131420,0.227282,0.528115,0.526254,0.600312,PITT
4,0.286560,4.214082,0.550083,427.042389,3.808350,3.839143,3.841085,3.744823,0.162421,0.505201,...,1000.004150,162584.0,900.433481,1069.912750,50.874363,0.195150,0.543591,0.531606,0.603308,PITT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,0.428731,3.030323,0.789654,2519.999512,3.176760,3.166740,3.359990,3.003550,0.169851,0.424819,...,1000.034668,241117.0,902.529590,1088.244409,56.683868,0.162535,0.476992,0.536843,0.537140,SBL
1097,0.610845,2.155928,0.800116,1769.720093,3.209497,3.164760,3.381280,3.082450,0.170732,0.405536,...,1000.039429,251136.0,903.951080,1093.323273,57.789230,0.193376,0.465232,0.545695,0.564010,SBL
1098,0.461773,2.794299,0.789859,2248.858398,3.149920,3.112220,3.326700,3.010840,0.165501,0.441190,...,1000.036438,209298.0,891.934216,1093.973322,61.108639,0.198508,0.497137,0.523571,0.564865,SBL
1099,0.457718,2.862913,0.706924,114.865364,3.486750,3.421200,3.881950,3.157100,0.209701,0.381839,...,999.990356,234957.0,904.907922,1101.429980,60.045422,0.235618,0.477310,0.563352,0.534626,MAX_MUN


## Cross-validation of the default classifier
Let's cross-validate the performance of our classifier using a Leave-one-site-out strategy.

In [6]:
# Define a splitting strategy
outer_cv = split.LeavePSitesOut(1, robust=True)
# Initiate the default model
model = init_pipeline()

We can now feed the model into the cross-validation loop:

In [7]:
score = cross_val_score(
    model,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    n_jobs=16,
)

After one or two minutes, the scores have been caculated for each of the 14 folds our splitter created.
The average performance is AUC=0.885.

In [8]:
print(score)
score.mean()

[0.9379845  0.98253968 0.97278912 0.89937107 0.89230769 0.83490566
 1.         0.94188862 0.48648649 1.         0.79148936 0.76294737
 0.88544892 1.        ]


0.8848684620750336

## Evaluating on held-out dataset

In [14]:
model = model.fit(
    X=train_x,
    y=train_y,
)

## Nested cross-validation

In [ ]:
p_grid = [{
    "scale__unit_variance": [True, False],
    "scale__with_centering": [True, False],
    "site_pred__disable": [False, True],
    "winnow__disable": [False, True],
    "svc__kernel": ["rbf"],
    "svc__C": [10],
    "svc__gamma": [0.1],
}]

In [ ]:
# Nested CV with parameter optimization
inner_cv = split.LeavePSitesOut(1, robust=True)
inner_cv.get_n_splits(X=train_x, y=train_y)

clf = GridSearchCV(
    estimator=pipe,
    param_grid=p_grid,
    cv=inner_cv,
    n_jobs=30,
    scoring="roc_auc",
)
# clf.fit(train_x, y=train_y)

In [ ]:
nested_score = cross_val_score(
    clf,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    verbose=10,
    n_jobs=16,
)
nested_score.mean()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_